# `ApJdataFrames` 
Grankin et al. 2008
---
`Title`: Results of the ROTOR-program     
`Authors`: Grankin et al.  


Data is from this paper:  
http://www.aanda.org/articles/aa/full/2008/09/aa8476-07/aa8476-07.html


In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
from astropy.io import ascii, votable, misc

### Download Data

In [4]:
#! mkdir ../data/Grankin08

In [5]:
#! wget -q -O ../data/Grankin08/table1_orig.tex http://www.aanda.org/articles/aa/full/2008/09/aa8476-07/table1.tex 

In [6]:
#! wget -q -O ../data/Grankin08/table3.tex http://www.aanda.org/articles/aa/full/2008/09/aa8476-07/table3.tex \

In [7]:
! ls ../data/Grankin08/

phot            table1_mod.tex  table1_plus.csv table3.tex
table1.csv      table1_orig.tex table3.csv


Ew, it's in $\LaTeX$ format!

## Table 1

In [8]:
#! head -n 30 ../data/Grankin08/table1.tex

### Two problems from latex.
1. The values with binaries go onto a new line, which foils data reading.
2. The `\farcs` latex screws up the decimal point.

To deal with these I had to manually delete (horror!) the carriage returns with Sublime Text, then do this:

In [9]:
#! cp ../data/Grankin08/table1_orig.tex ../data/Grankin08/table1_mod.tex

In [10]:
#! sed -i 's/\\farcs/./g' ../data/Grankin08/table1_mod.tex

In [11]:
names_1 = ['Name', 'HBC', 'SpT', 'JD_min_max', 'N_seasons', 'V_range', 'N_obs', 'avgB_V', 'avgV_R', 'mult', 'ref']

In [12]:
tab1 = pd.read_csv('../data/Grankin08/table1_mod.tex', sep='&',
                   skiprows=10, names=names_1, engine='python', skipfooter=8)

In [13]:
tab1.tail()

,Name,HBC,SpT,JD_min_max,N_seasons,V_range,N_obs,avgB_V,avgV_R,mult,ref
43,Wa CrA/2,678,G8 IV,48049--48100,1,10.69--10.53,28,0.85,0.74,,\\
44,Wa Oph/1,630,K2 IV,49145--53228,8,12.19--11.84,190,1.38,1.30,,\\
45,Wa Oph/2,633,K1 IV,51733--53228,5,11.77--11.60,101,1.16,1.11,,\\
46,Wa Oph/3,634,K0 IV,49145--53228,9,10.98--10.73,255,1.19,1.07,,\\
47,Wa Oph/4,652,K4,49147--52059,4,13.72--10.44,104,1.88,1.83,VB(8.7),re93 \\


In [14]:
tab1.to_csv('../data/Grankin08/table1.csv', index=False)

## Table 3

In [15]:
#! tail -n 15 ../data/Grankin08/table3.tex

In [16]:
names = ['Name', 'Epochs', 'delta_V_min', 'delta_V_max', 'HJD0-24000000', 'Period', 'Ref1', 'Ref2']

In [17]:
tab3 = pd.read_csv('../data/Grankin08/table3.tex', sep='&', comment='\\',
                   skiprows=10, names=names, engine='python', skipfooter=5)

In [18]:
tab3.tail()

,Name,Epochs,delta_V_min,delta_V_max,HJD0-24000000,Period,Ref1,Ref2
31,VY Tau,1985--2001,0.11,0.31,44610.7,5.36995,gr91,gr94
32,Wa CrA/1,1990,0.32,0.32,48048.3,2.24000,sh95,sh95
33,Wa CrA/2,1990,0.16,0.16,48048.3,2.79000,co92,sh95
34,Wa Oph/1,1993--2004,0.10,0.30,48986.1,3.79200,za93,this paper
35,Wa Oph/3,1993--2004,0.06,0.23,49144.3,1.52140,sh98,this paper


In [19]:
tab3.to_csv('../data/Grankin08/table3.csv', index=False)

## Raw data files

Copied from Vizier and pasted into a text document...

In [20]:
#! head ../data/Grankin08/grankin08_dat_files.txt

In [21]:
#gr_dat = pd.read_csv('../data/Grankin08/grankin08_dat_files.txt', usecols=[0],
#                     delim_whitespace=True, names=['filename'])

Download the data:

In [22]:
import os

Only need to run this once:

```python
for i in range(len(gr_dat)):
#for i in range(3):
    fn = gr_dat.filename[i]
    web_addr = 'http://vizier.cfa.harvard.edu/vizier/ftp/cats/J/A+A/479/827/phot/'
    cmd = 'curl '+ web_addr + fn +' > ' + '../data/Grankin08/phot/'+fn 
    os.system(cmd)
    print(cmd)
```

In [23]:
#! gzip -d ../data/Grankin08/phot/*.dat.gz

In [24]:
#gr_dat['dat_fn'] = gr_dat.filename.str[0:-3]

## Add RA, DEC, and other simbad info

In [25]:
from astroquery.simbad import Simbad

In [26]:
gr_t1 = tab1

In [27]:
gr_t1['HBC_name'] = 'HBC' + gr_t1.HBC

In [28]:
gr_t1['alt_name'] = gr_t1.HBC_name

In [29]:
gr_t1.alt_name[18] = 'TAP 10'
gr_t1.alt_name[19] = 'TAP 11'
gr_t1.alt_name[20] = 'TAP 14'

In [30]:
Simbad.add_votable_fields('sptype', 'otype')

In [31]:
gr_t1['pref_name'] = ''
gr_t1['RA'] = ''
gr_t1['DEC'] = ''
gr_t1['SpT_simbad'] = ''
gr_t1['Otype_simbad'] = ''

In [32]:
N_sources = len(gr_t1)

#### You only have to run this once:

In [33]:
for i in range(N_sources):
    name = gr_t1.Name[i]
    name_alt = gr_t1.alt_name[i]
    result_table = Simbad.query_object(name)
    try:
        RA, DEC = result_table['RA'].data.data[0], result_table['DEC'].data.data[0]
        SpT, Otype = result_table['SP_TYPE'].data.data[0], result_table['OTYPE'].data.data[0]
        print("{} was found in Simbad.".format(name))
    except TypeError:
        print("Attempt 1 did not work for {}, trying HBC name: {}...".format(name, name_alt), end='')
        result_table = Simbad.query_object(name_alt)
        RA, DEC = result_table['RA'].data.data[0], result_table['DEC'].data.data[0]
        SpT, Otype = result_table['SP_TYPE'].data.data[0], result_table['OTYPE'].data.data[0]
        print(' success!')
        name = name_alt
    gr_t1 = gr_t1.set_value(i, 'pref_name', name)
    gr_t1 = gr_t1.set_value(i, 'RA', RA)
    gr_t1 = gr_t1.set_value(i, 'DEC', DEC)
    gr_t1 = gr_t1.set_value(i, 'SpT_simbad', SpT.decode())
    gr_t1 = gr_t1.set_value(i, 'Otype_simbad', Otype.decode())

Attempt 1 did not work for Anon 1    , trying HBC name: HBC 366 ... success!
HD 283572  was found in Simbad.
Attempt 1 did not work for Hubble 4  , trying HBC name: HBC 374 ... success!
LkCa 1     was found in Simbad.
LkCa 2     was found in Simbad.
LkCa 3     was found in Simbad.
LkCa 4     was found in Simbad.
LkCa 5     was found in Simbad.
LkCa 7     was found in Simbad.
LkCa 11    was found in Simbad.
LkCa 14    was found in Simbad.
LkCa 16    was found in Simbad.
LkCa 19    was found in Simbad.
LkCa 21    was found in Simbad.
Attempt 1 did not work for SR 9      , trying HBC name: HBC 264 ... success!
SR 12      was found in Simbad.
TAP 4      was found in Simbad.
TAP 9      was found in Simbad.
Attempt 1 did not work for TAP 10AB , trying HBC name: TAP 10... success!
Attempt 1 did not work for TAP 11AB , trying HBC name: TAP 11... success!
Attempt 1 did not work for TAP 14NE , trying HBC name: TAP 14... success!
TAP 26     was found in Simbad.
TAP 35     was found in Simbad.
TAP

```python
for i in range(N_sources):
    name = gr_t1.Name[i]
    name_alt = gr_t1.alt_name[i]
    result_table = Simbad.query_object(name)
    try:
        RA, DEC = result_table['RA'].data.data[0], result_table['DEC'].data.data[0]
        SpT, Otype = result_table['SP_TYPE'].data.data[0], result_table['OTYPE'].data.data[0]
        print("{} was found in Simbad.".format(name))
    except TypeError:
        print("Attempt 1 did not work for {}, trying HBC name: {}...".format(name, name_alt), end='')
        result_table = Simbad.query_object(name_alt)
        RA, DEC = result_table['RA'].data.data[0], result_table['DEC'].data.data[0]
        SpT, Otype = result_table['SP_TYPE'].data.data[0], result_table['OTYPE'].data.data[0]
        print(' success!')
        name = name_alt
    gr_t1 = gr_t1.set_value(i, 'pref_name', name)
    gr_t1 = gr_t1.set_value(i, 'RA', RA)
    gr_t1 = gr_t1.set_value(i, 'DEC', DEC)
    gr_t1 = gr_t1.set_value(i, 'SpT_simbad', SpT.decode())
    gr_t1 = gr_t1.set_value(i, 'Otype_simbad', Otype.decode())
```

In [34]:
gr_t1.tail()

,Name,HBC,SpT,JD_min_max,N_seasons,V_range,N_obs,avgB_V,avgV_R,mult,ref,HBC_name,alt_name,pref_name,RA,DEC,SpT_simbad,Otype_simbad
43,Wa CrA/2,678,G8 IV,48049--48100,1,10.69--10.53,28,0.85,0.74,,\\,HBC 678,HBC 678,HBC 678,19 02 01.9844,-37 07 43.538,G5e,Orion_V*
44,Wa Oph/1,630,K2 IV,49145--53228,8,12.19--11.84,190,1.38,1.30,,\\,HBC 630,HBC 630,HBC 630,16 11 08.908,-19 04 46.86,K2IV(e),TTau*
45,Wa Oph/2,633,K1 IV,51733--53228,5,11.77--11.60,101,1.16,1.11,,\\,HBC 633,HBC 633,HBC 633,16 11 59.272,-19 06 53.36,K2Ve,TTau*
46,Wa Oph/3,634,K0 IV,49145--53228,9,10.98--10.73,255,1.19,1.07,,\\,HBC 634,HBC 634,HBC 634,16 12 40.5054,-18 59 28.137,K0IV(e),TTau*
47,Wa Oph/4,652,K4,49147--52059,4,13.72--10.44,104,1.88,1.83,VB(8.7),re93 \\,HBC 652,HBC 652,HBC 652,16 48 18.00,-14 11 15.9,K5Ve,TTau*


In [35]:
gr_t1.to_csv('../data/Grankin08/table1_plus.csv', index=False)

## The end!